# GENERAZIONI PREDIZIONI SUL DATASET DI TEST

In [49]:
import torch
import numpy as np
import cv2
from PIL import Image
from torchvision import transforms , models
import matplotlib.pyplot as plt
from CableDetection import CableTrainDataset
from torch.utils.data import DataLoader
from pycocotools import mask as coco_mask
import json
import segmentation_models_pytorch as smp
import torch.nn.functional as F

## IMPOSTAZIONE DEI PERCORSI DEL PROGETTO

In questa sezione vengono definiti i percorsi principali del progetto.

In [50]:
img_directory="dataset/test/"
img_json_path="dataset/test/test.json"
file_json_predictions="output/matricola.json"
model_path = "models/UNET++_final.pth"
size_img=(704,704)  

## DEFINIZIONE DEL DEVICE

In [51]:
DEVICE = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

## DEFINIZIONE MODELLO

In [52]:
NUM_CLASSES = 1

model = smp.UnetPlusPlus(
    encoder_name="timm-resnest50d", 
    encoder_weights="imagenet",
    in_channels=3,
    classes=NUM_CLASSES,
    decoder_attention_type="scse", 
)

## CARICAMENTO DEI PESI DEL MODELLO ADDESTRATO

In [53]:
model.to(DEVICE)
model.load_state_dict(torch.load(model_path, map_location= DEVICE))
model.eval()

UnetPlusPlus(
  (encoder): TimmUniversalEncoder(
    (model): FeatureListNet(
      (conv1): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): ResNestBottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

## DEFINIZIONE DEL DATASET DI TEST

In [54]:
testset=CableTrainDataset(img_dir=img_directory, image_json_path=img_json_path,size=size_img)
dataloader= DataLoader(testset, batch_size=1, shuffle=False)

##  ESTRAZIONE DELLE METRICHE 

Calcola metriche per ogni oggetto segmentato in una maschera:  
- Segmentazione RLE compatibile COCO  
- Bounding box e area  
- Punteggio medio dei pixel  
- Linee polari tramite Hough Transform  
- Restituisce i risultati filtrati per area minima


In [55]:
def calculate_metrics(mask,image_id,probs):
    results=[]
    num_labels, labels_im = cv2.connectedComponents(mask)
    for i in range(1, num_labels):
        blob_mask = (labels_im == i).astype(np.uint8)
            
        # 1. Segmentazione RLE
        # La funzione encode accetta una maschera Fortran-contiguous
        rle = coco_mask.encode(np.asfortranarray(blob_mask))
            
        # RLE è di tipo bytes, deve essere convertito in stringhe compatibili con JSON (utf-8)
        rle['counts'] = rle['counts'].decode('utf-8')

        # 2. Bounding box e Area
        ys, xs = np.where(blob_mask)
        if len(xs) == 0:
            continue # Salta maschere vuote

        x1, y1 = xs.min(), ys.min()
        x2, y2 = xs.max(), ys.max()
        bbox = [float(x1), float(y1), float(x2 - x1), float(y2 - y1)] # Formato COCO: [x, y, w, h]
        area = int(np.sum(blob_mask))

        # 3. Punteggio medio dei pixel
        score = probs[ys, xs].mean()

        # 4. Coordinate polari con HoughLines
        # Nota: HoughLines vuole una maschera in formato CV_8UC1 (np.uint8)
        mask_uint8 = blob_mask * 255 
        lines = cv2.HoughLines(mask_uint8, rho=1, theta=np.pi/180, threshold=20)
        
        polar_lines = []
        if lines is not None and len(lines) > 0:
            rho, theta = lines[0][0]
            # Converti in float standard per JSON
            polar_lines = [float(rho), float(theta)]

        id_univoco=0
        # 5. Salva risultati in formato COCO
        metrics_dict = {
            "image_id": image_id.item(),
            "category_id": 0, # Assumendo che 'cavo' sia category_id 0
            "bbox": bbox,
            "segmentation": rle, # Formato RLE
            "score": float(score),
            "lines": polar_lines, # Corrisponde a POLAR_COORDINATE
            "area": area,
            "height": 700,
            "width": 700,
            "id": id_univoco # ID univoco per la predizione
        }
        if(area>50):
            results.append(metrics_dict)
    return results

## GENERAZIONE DELLE PREDIZIONI SU TESTSET

- Per ogni immagine:
  - Si calcolano le probabilità tramite il modello (`torch.sigmoid`)  
  - Si genera la maschera binaria con soglia maggiore di 0.85  
  - Si estraggono le metriche tramite `calculate_metrics`
- I risultati vengono salvati in formato JSON per valutazioni successive.


In [56]:
results_list=[]

for image, _ ,image_id in dataloader:
    image = image.to(DEVICE)
    with torch.no_grad():
        output = model(image)
        output_resized = F.interpolate(output, size=(700,700), mode='bilinear', align_corners=False)
        probs = torch.sigmoid(output_resized)[0,0].cpu().numpy()  # probabilità cavo
        mask = (probs > 0.9).astype(np.uint8)  # mask binaria 0/1

        result= calculate_metrics(mask,image_id,probs)
        results_list.extend(result)

with open(file_json_predictions, "w") as f:
    json.dump(results_list, f)